In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats

In [59]:
path="/Users/guoyalong/Downloads/injuries_2010-2020.csv"
path2="/Users/guoyalong/Downloads/Team_Records.csv"

In [61]:
#read in Team_records file
Team_records=pd.read_csv(path2)
Team_records.head()

,Season,Lg,Team,W,L,W/L%,Finish,SRS,Pace,Rel_Pace,ORtg,Rel_ORtg,DRtg,Rel_DRtg,Playoffs,Coaches,Top WS
0,2017-18,NBA,Boston Celtics,29,10,0.744,1,4.38,95.4,-1.7,108.0,0.2,102.8,-5.0,NaN,B. Stevens (29-10),K. Irving (5.7)
1,2016-17,NBA,Boston Celtics*,53,29,0.646,1,2.25,96.8,0.4,111.2,2.4,108.4,-0.4,Lost E. Conf. Finals,B. Stevens (53-29),I. Thomas (12.5)
2,2015-16,NBA,Boston Celtics*,48,34,0.585,2,2.84,98.5,2.7,106.8,0.4,103.6,-2.8,Lost E. Conf. 1st Rnd.,B. Stevens (48-34),I. Thomas (9.7)
3,2014-15,NBA,Boston Celtics*,40,42,0.488,2,-0.40,95.8,1.9,104.7,-0.9,104.5,-1.1,Lost E. Conf. 1st Rnd.,B. Stevens (40-42),T. Zeller (6.5)
4,2013-14,NBA,Boston Celtics,25,57,0.305,4,-4.97,93.3,-0.6,102.9,-3.8,107.7,1.0,NaN,B. Stevens (25-57),B. Bass (5.1)


In [3]:
#read in injuries csv file
injuries_2010_2020=pd.read_csv(path)
injuries_2010_2020.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [62]:
##Clean up injuries data

In [4]:
#drop null values for acquired
acquired_df=injuries_2010_2020.dropna(subset=["Acquired"])
acquired_df

,Date,Team,Acquired,Relinquished,Notes
53,2010-10-27,Heat,Jerry Stackhouse,NaN,activated from IL
81,2010-10-27,Rockets,Jermaine Taylor,NaN,activated from IL
101,2010-10-29,Cavaliers,Samardo Samuels,NaN,activated from IL
103,2010-10-29,Celtics,Luke Harangody,NaN,activated from IL
105,2010-10-29,Grizzlies,Marc Gasol,NaN,activated from IL
...,...,...,...,...,...
27091,2020-09-08,Bucks,Giannis Antetokounmpo,NaN,returned to lineup
27094,2020-09-12,Lakers,JaVale McGee,NaN,returned to lineup
27100,2020-09-30,Lakers,Dion Waiters,NaN,activated from IL
27103,2020-10-02,Heat,Chris Silva,NaN,activated from IL


In [8]:
#get unique number of players that were returning to the lineup
acquired_df["Acquired"].nunique()

1111

In [53]:
#get unique number of teams
acquired_df["Team"].nunique()

32

In [71]:
#get names of teams
acquired_df["Team"].unique()

array(['Heat', 'Rockets', 'Cavaliers', 'Celtics', 'Grizzlies', 'Hawks',
       'Kings', 'Raptors', 'Warriors', 'Pacers', 'Spurs', 'Wizards',
       'Clippers', 'Blazers', 'Hornets', 'Nets', 'Timberwolves', 'Knicks',
       'Lakers', 'Magic', '76ers', 'Thunder', 'Mavericks', 'Bobcats',
       'Pistons', 'Suns', 'Nuggets', 'Bucks', 'Bulls', 'Jazz', 'Pelicans',
       'Bullets'], dtype=object)

In [51]:
#drop null values for relinquished
relinquished_df=injuries_2010_2020.dropna(subset=["Relinquished"])
relinquished_df

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...,...
27097,2020-09-22,Celtics,NaN,Romeo Langford,surgery on right wrist (out for season)
27098,2020-09-23,Heat,NaN,Gabe Vincent,sore right knee (DTD)
27099,2020-09-30,Heat,NaN,Bam Adebayo,strained left shoulder (DTD)
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD)


In [54]:
#get unique number of players missing a game or placed on inactive list
relinquished_df["Relinquished"].nunique()

1156

In [55]:
# get total number of relinquished dates
relinquished_df["Relinquished"].count()

17560

In [ ]:
###Clean up Team_records Data

In [72]:
clean_team_records=Team_records.loc[:,["Season","Team","W","L","W/L%","Finish"]]
clean_team_records

,Season,Team,W,L,W/L%,Finish
0,2017-18,Boston Celtics,29,10,0.744,1
1,2016-17,Boston Celtics*,53,29,0.646,1
2,2015-16,Boston Celtics*,48,34,0.585,2
3,2014-15,Boston Celtics*,40,42,0.488,2
4,2013-14,Boston Celtics,25,57,0.305,4
...,...,...,...,...,...,...
1478,1999-00,Vancouver Grizzlies,22,60,0.268,7
1479,1998-99,Vancouver Grizzlies,8,42,0.160,7
1480,1997-98,Vancouver Grizzlies,19,63,0.232,6
1481,1996-97,Vancouver Grizzlies,14,68,0.171,7
